Обрати один з методів побудови індексу:

       MapReduce

Та побудувати індекс великої колекції.

In [1]:
import os

data_folders = ['generated', 'data']
data_folder_path = os.path.join('.', data_folders[0])
books_list = os.listdir(data_folder_path)
books_list

['g2_1146.txt',
 'g2_0.txt',
 'g2_2329.txt',
 'g2_580.txt',
 'g2_35.txt',
 'g2_1620.txt',
 'g2_1634.txt',
 'g2_594.txt',
 'g2_21.txt',
 'g2_1152.txt',
 'g2_2473.txt',
 'g2_2315.txt',
 'g2_2301.txt',
 'g2_1608.txt',
 'g2_2467.txt',
 'g2_225.txt',
 'g2_1185.txt',
 'g2_543.txt',
 'g2_557.txt',
 'g2_1191.txt',
 'g2_231.txt',
 'g2_2498.txt',
 'g2_219.txt',
 'g2_1807.txt',
 'g2_1813.txt',
 'g2_1422.txt',
 'g2_782.txt',
 'g2_2895.txt',
 'g1_58.txt',
 'g2_1344.txt',
 'g2_1350.txt',
 'g2_2659.txt',
 'g2_2881.txt',
 'g2_796.txt',
 'g2_1436.txt',
 'g2_966.txt',
 'g2_2117.txt',
 'g1_64.txt',
 'g2_1378.txt',
 'g2_2671.txt',
 'g2_2665.txt',
 'g1_70.txt',
 'g2_972.txt',
 'g2_2103.txt',
 'g2_741.txt',
 'g2_999.txt',
 'g2_1387.txt',
 'g2_2856.txt',
 'g2_2842.txt',
 'g2_1393.txt',
 'g2_755.txt',
 'g2_769.txt',
 'g2_1556.txt',
 'g2_190.txt',
 'g2_2739.txt',
 'g2_1230.txt',
 'g2_1224.txt',
 'g2_184.txt',
 'g2_1542.txt',
 'g2_2063.txt',
 'g2_812.txt',
 'g2_2705.txt',
 'g2_2711.txt',
 'g2_1218.txt',
 'g2_20

In [6]:
from pympler.asizeof import asizeof


#  Maximum expected memory usage in MB
MAX_MEMORY = 500

def enought_memory(*args):
    total_memory_usage = 0
    for arg in args:
        total_memory_usage += asizeof(arg) / 1024 / 1024
    print(total_memory_usage)
    if total_memory_usage > MAX_MEMORY:
        return False
    return True

enought_memory()

0


True

## Map

In [8]:
import codecs
import re
import shutil
import os, psutil

shutil.rmtree('./mapped/')
os.mkdir('./mapped')

MAX_BYTES = 3000 * 1024 * 1024

process = psutil.Process(os.getpid())

i = 0
while i < len(books_list):
    print('\t---\t---\t---')
    index = dict()

    while process.memory_info().rss < MAX_BYTES:# enought_memory(i, books_list, index):
        if i >= len(books_list):
            break

        book = books_list[i]
        book_path = os.path.join(data_folder_path, book)

        with codecs.open(book_path, "r", "utf_8_sig") as fileObj:
            text = fileObj.read()

        words = re.findall("[a-z]+[’'-]?[a-z]*", text.lower())
        
        words_count = dict()
        for word in words:
            if word in words_count:
                words_count[word] += 1
            else:
                words_count[word] = 1
        
        for word, freq in words_count.items():
            if word in index:
                index[word] += [book, freq]
            else:
                index[word] = [book, freq]
        
        print(i, book_path, process.memory_info().rss / 1024**2)
        i += 1
    
    print('writing start')
    with open(f'./mapped/map_{i}', 'w') as f:
        for key in sorted(index):
            f.write(f"{key} {' '.join([str(v) for v in index[key]])}\n")
    print('writing end')

	---	---	---
0 ./generated/g2_1146.txt 756.3828125
1 ./generated/g2_0.txt 766.92578125
2 ./generated/g2_2329.txt 803.85546875
3 ./generated/g2_580.txt 775.32421875
4 ./generated/g2_35.txt 810.3984375
5 ./generated/g2_1620.txt 828.11328125
6 ./generated/g2_1634.txt 783.5625
7 ./generated/g2_594.txt 773.41796875
8 ./generated/g2_21.txt 816.875
9 ./generated/g2_1152.txt 848.96484375
10 ./generated/g2_2473.txt 862.6640625
11 ./generated/g2_2315.txt 859.90625
12 ./generated/g2_2301.txt 837.09765625
13 ./generated/g2_1608.txt 859.1328125
14 ./generated/g2_2467.txt 884.1484375
15 ./generated/g2_225.txt 854.765625
16 ./generated/g2_1185.txt 822.95703125
17 ./generated/g2_543.txt 839.92578125
18 ./generated/g2_557.txt 813.33203125
19 ./generated/g2_1191.txt 785.3515625
20 ./generated/g2_231.txt 808.44140625
21 ./generated/g2_2498.txt 825.92578125
22 ./generated/g2_219.txt 832.01953125
23 ./generated/g2_1807.txt 861.484375
24 ./generated/g2_1813.txt 841.8046875
25 ./generated/g2_1422.txt 851.144

KeyboardInterrupt: 

## Reduce

In [42]:
# shutil.rmtree('./reduced/')
# os.mkdir('./reduced')

mapped_files = os.listdir('./mapped')
try:
    data = []
    readers = [open(f'./mapped/{f}') for f in mapped_files]
    for r in readers:
        data.append(r.readline().strip().split())
    target = open('./reduced/reduce_1', 'w')

    print(data)

    docs_count = len(readers)
    while any(data):
        if not all(data):
            new_readers, new_data = [], []
            for j in range(docs_count):
                if data[j]:
                    new_readers.append(readers[j])
                    new_data.append(data[j])
                else:
                    readers[j].close()
            readers, data, docs_count = new_readers, new_data, len(new_data)
            print(data)

        min_word = min((d[0] for d in data))
        reduced = []
        for i in range(docs_count):
            if data[i][0] == min_word:
                reduced += data[i][1:]
                data[i] = readers[i].readline().strip().split()
        target.write(f"{min_word} {' '.join([str(v) for v in reduced])}\n")

except Exception as e:
    print(e)
finally:
    target.close()
    for r in readers:
        r.close()

[['a', 'great-expectations.txt', '3992'], ['a', 'frankenstein.txt', '1392', '1984.txt', '2572', 'the-great-gatsby.txt', '1408', 'grimms-fairy-tales.txt', '1911', 'wuthering-heights.txt', '2317', 'tender-is-the-night.txt', '2906'], ['a', 'the-adventures-of-huckleberry-finn.txt', '2927', 'pride-and-prejudice.txt', '1953', 'anna-karenina.txt', '6172', 'jane-eyre.txt', '4463']]
[['zig-zagging', '1984.txt', '1'], ['zhivahov', 'anna-karenina.txt', '1']]
[['zug', 'tender-is-the-night.txt', '2']]


## Version 2

In [3]:
import codecs
import re
import shutil
import os, psutil

shutil.rmtree('./mapped/')
os.mkdir('./mapped')

MAX_BYTES = 3000 * 1024 * 1024

process = psutil.Process(os.getpid())

i = 0
while i<40 or i < len(books_list):
    print('\t---\t---\t---')
    mapped = []
    
    while process.memory_info().rss < MAX_BYTES: #enought_memory(i, books_list, mapped):
        if i >= len(books_list):
            break

        book = books_list[i]
        book_path = os.path.join(data_folder_path, book)

        with codecs.open(book_path, "r", "utf_8_sig") as fileObj:
            text = fileObj.read()

        words = re.findall("[a-z]+[’'-]?[a-z]*", text.lower())
        mapped += [[f'{word} {book}', 1] for word in words]
        
        print(i, book_path, process.memory_info().rss / 1024**2)
        i += 1
    
    LOCAL_REDUCE = True
    if LOCAL_REDUCE:
        print('\tlocal reduce start')
        with open(f'./mapped/map_{i}', 'w') as f:
            prev_el, count = '', 0
            for el in sorted(mapped, key=lambda x: x[0]):
                if prev_el == el[0]:
                    count += el[1]
                else:
                    f.write(f"{prev_el} {str(count)}\n") if prev_el else 0
                    prev_el, count = el[0], 1
        print('\tlocal reduce end')
    else:
        print('\tsorting and writing start')
        with open(f'./mapped/map_{i}', 'w') as f:
            for el in sorted(mapped, key=lambda x: x[0]):
                f.write(f"{' '.join([str(v) for v in el])}\n")
        print('\tsorting and writing end')

	---	---	---
0 ./generated/g2_1146.txt 186.60546875
1 ./generated/g2_0.txt 267.70703125
2 ./generated/g2_2329.txt 350.16796875
3 ./generated/g2_580.txt 448.0234375
4 ./generated/g2_35.txt 571.890625
5 ./generated/g2_1620.txt 801.19921875
6 ./generated/g2_1634.txt 853.43359375
7 ./generated/g2_594.txt 815.68359375
8 ./generated/g2_21.txt 1017.609375
9 ./generated/g2_1152.txt 1182.07421875
10 ./generated/g2_2473.txt 1387.09375
11 ./generated/g2_2315.txt 1473.0703125
12 ./generated/g2_2301.txt 1584.8359375
13 ./generated/g2_1608.txt 1621.90625
14 ./generated/g2_2467.txt 1868.8125
15 ./generated/g2_225.txt 1894.265625
16 ./generated/g2_1185.txt 2097.33984375
17 ./generated/g2_543.txt 2310.74609375
18 ./generated/g2_557.txt 2316.21484375
19 ./generated/g2_1191.txt 2301.02734375
20 ./generated/g2_231.txt 2571.9609375
21 ./generated/g2_2498.txt 2690.4921875
22 ./generated/g2_219.txt 2741.40625
23 ./generated/g2_1807.txt 2675.65234375
24 ./generated/g2_1813.txt 2551.76171875
25 ./generated/g2_

In [7]:
mapped = []

In [4]:
# shutil.rmtree('./reduced/')
# os.mkdir('./reduced')

mapped_files = os.listdir('./mapped')
try:
    data = []
    readers = [open(f'./mapped/{f}') for f in mapped_files]
    for r in readers:
        data.append(r.readline().strip().split())
    target = open('./reduced/reduce_generated_all_ds_combine.txt', 'w')
    min_word = prev_word = min((d[0] for d in data))

    print(data)

    docs_count = len(readers)
    while any(data):
        if not all(data):
            new_readers, new_data = [], []
            for j in range(docs_count):
                if data[j]:
                    new_readers.append(readers[j])
                    new_data.append(data[j])
                else:
                    readers[j].close()
            readers, data, docs_count = new_readers, new_data, len(new_data)
            print(data)

        reduced = dict()
        
        while min_word == prev_word:
            for i in range(docs_count):
                if not data[i]:
                    continue
                if data[i][0] == min_word:
                    if data[i][1] in reduced:
                        reduced[data[i][1]] += int(data[i][2])
                    else:
                        reduced[data[i][1]] = int(data[i][2])
                    data[i] = readers[i].readline().strip().split()
            min_word = min((d[0] for d in data if d)) if any(data) else ''
        
        target.write(f"{prev_word} {' '.join([k+' '+str(v) for k, v in reduced.items()])}\n")
        prev_word = min_word
    print(data)

except Exception as e:
    print(e)
    print(data)
    raise Exception from e
finally:
    target.close()
    for r in readers:
        r.close()

[['a', 'g2_1005.txt', '120'], ['a', 'g2_1157.txt', '134'], ['a', 'g2_1040.txt', '48'], ['a', 'g2_1253.txt', '50'], ['a', 'g2_100.txt', '44'], ['a', 'g2_1204.txt', '78'], ['a', 'g2_1068.txt', '73'], ['a', 'g2_1244.txt', '17'], ['a', 'g2_1306.txt', '58'], ['a', 'g2_101.txt', '93'], ['a', 'g2_1023.txt', '96'], ['a', 'g2_102.txt', '88'], ['a', 'g2_1209.txt', '58'], ['a', 'g2_105.txt', '119'], ['a', 'g2_1104.txt', '108'], ['a', 'g2_1046.txt', '126'], ['a', 'g2_1007.txt', '117'], ['a', 'g2_1262.txt', '56'], ['a', 'g2_1128.txt', '39'], ['a', 'g2_1009.txt', '155'], ['a', 'g2_1059.txt', '136'], ['a', 'g2_1348.txt', '116'], ['a', 'g2_1127.txt', '128'], ['a', 'g2_1268.txt', '131'], ['a', 'g2_1044.txt', '86'], ['a', 'g2_1223.txt', '47'], ['a', 'g2_1106.txt', '139'], ['a', 'g2_1200.txt', '84'], ['a', 'g2_1032.txt', '23'], ['a', 'g2_1102.txt', '52'], ['a', 'g2_1258.txt', '122'], ['a', 'g2_1226.txt', '27'], ['a', 'g2_1066.txt', '72'], ['a', 'g2_1231.txt', '45'], ['a', 'g2_1246.txt', '87'], ['a', 'g2_